In [2]:
from gurobipy import GRB
import gurobipy as gp

# === CONFIGURATION ===
cycle_times = [9.35, 5.6, 5.08, 8.53, 7.26, 8.12, 8.71, 4.17, 3.85, 4.46,
               7.03, 4.01, 5.08, 5.37, 6.05, 4.6, 9.22, 5.58, 7.43, 3.53,
               5.09, 4, 3.61, 3.05, 3.41]
n = len(cycle_times)
total_ops = 34
shift_hours = 9
epsilon = 0.05

status_names = {
    1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD',
    5: 'UNBOUNDED', 6: 'CUTOFF', 7: 'ITERATION_LIMIT', 8: 'NODE_LIMIT',
    9: 'TIME_LIMIT', 10: 'SOLUTION_LIMIT', 11: 'INTERRUPTED',
    12: 'NUMERIC', 13: 'SUBOPTIMAL', 14: 'INPROGRESS', 15: 'USER_OBJ_LIMIT'
}

# === PHASE 1: MAXIMIZE AVERAGE THROUGHPUT ===
def build_phase1_model():
    """
    Builds Phase 1 model to maximize average throughput, respecting group constraints.
    """
    m = gp.Model("MaxThroughput")
    m.Params.OutputFlag = 1

    mu = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="mu")
    x = m.addVars(total_ops, n, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="x")
    z = m.addVars(total_ops, n, vtype=GRB.BINARY, name="z")
    throughput = m.addVars(n, lb=0, vtype=GRB.CONTINUOUS, name="throughput")
    g1 = m.addVars(total_ops, vtype=GRB.BINARY, name="g1")  # Group 1 (stations 1-10)
    g2 = m.addVars(total_ops, vtype=GRB.BINARY, name="g2")  # Group 2 (stations 11-25)

    # Assignment constraints
    for j in range(total_ops):
        m.addConstr(gp.quicksum(x[j, i] for i in range(n)) == 1, name=f"assign_sum_{j}")
        m.addConstr(gp.quicksum(z[j, i] for i in range(n)) <= 2, name=f"max_stations_{j}")
        m.addConstr(g1[j] + g2[j] == 1, name=f"group_exclusive_{j}")  # Operator in one group
        for i in range(n):
            m.addConstr(x[j, i] <= z[j, i], name=f"x_z_link_{j}_{i}")
            m.addConstr(x[j, i] >= z[j, i] * epsilon, name=f"x_z_min_{j}_{i}")
            if i < 10:
                m.addConstr(z[j, i] <= g1[j], name=f"group1_limit_{j}_{i}")
            else:
                m.addConstr(z[j, i] <= g2[j], name=f"group2_limit_{j}_{i}")

    # Station assignment limit
    for i in range(n):
        m.addConstr(gp.quicksum(z[j, i] for j in range(total_ops)) <= 2, name=f"max_ops_station_{i}")

    # Throughput calculation
    for i in range(n):
        m.addConstr(throughput[i] == gp.quicksum(60 * x[j, i] / cycle_times[i] for j in range(total_ops)),
                    name=f"throughput_{i}")

    # Average throughput
    m.addConstr(mu == gp.quicksum(throughput[i] for i in range(n)) / n, name="avg_throughput")
    m.setObjective(mu, GRB.MAXIMIZE)
    return m, mu, x, z, throughput, g1, g2

def run_phase1():
    m1, mu, x, z, throughput, g1, g2 = build_phase1_model()
    try:
        m1.optimize()
    except gp.GurobiError as e:
        print(f"Gurobi error in Phase 1: {e}")
        exit(1)

    if m1.status == GRB.INFEASIBLE:
        m1.computeIIS()
        m1.write("infeasible_phase1.ilp")
        print("Phase 1 Infeasible. Check infeasible_phase1.ilp")
        exit(1)
    elif m1.status != GRB.OPTIMAL:
        print(f"Phase 1 failed with status {m1.status} ({status_names.get(m1.status, 'UNKNOWN')})")
        exit(1)

    return mu.X, m1, x, z, g1, g2

# === PHASE 2: MINIMIZE DEVIATION WHILE MAINTAINING THROUGHPUT ===
def extract_warm_start(m1, x, z, g1, g2):
    """
    Extracts warm-start data from Phase 1, ensuring group constraints are respected.
    """
    data = []
    print("\n=== Warm-Start Data Validation ===")
    for j in range(total_ops):
        assigned_stations = [i for i in range(n) if z[j, i].X > 0.5]
        group1 = g1[j].X > 0.5
        group2 = g2[j].X > 0.5
        if not assigned_stations:
            print(f"Operator {j+1}: No assignments")
            continue
        if group1 and any(i >= 10 for i in assigned_stations):
            print(f"Warning: Operator {j+1} in group 1 but assigned to stations {assigned_stations}")
        if group2 and any(i < 10 for i in assigned_stations):
            print(f"Warning: Operator {j+1} in group 2 but assigned to stations {assigned_stations}")
        total_x = sum(x[j, i].X for i in range(n))
        if abs(total_x - 1) > 1e-6:
            print(f"Warning: Operator {j+1} assignment sum = {total_x:.6f}, expected 1")
        for i in assigned_stations:
            xv, zv = x[j, i].X, z[j, i].X
            if xv >= epsilon:
                data.append((j, i, xv, zv))
        print(f"Operator {j+1}: Stations {assigned_stations}, x_sum = {total_x:.6f}, Group = {'1' if group1 else '2'}")
    return data

def build_phase2_model(mu_star, warm_data):
    m = gp.Model("BalanceWorkload")
    m.Params.OutputFlag = 1
    m.Params.TimeLimit = 180
    m.Params.MIPGap = 0.001
    m.Params.MIPFocus = 2
    m.Params.Heuristics = 0.1

    x = m.addVars(total_ops, n, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="x")
    z = m.addVars(total_ops, n, vtype=GRB.BINARY, name="z")
    throughput = m.addVars(n, lb=0, vtype=GRB.CONTINUOUS, name="throughput")
    deviation = m.addVars(n, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="deviation")
    g1 = m.addVars(total_ops, vtype=GRB.BINARY, name="g1")
    g2 = m.addVars(total_ops, vtype=GRB.BINARY, name="g2")

    for j in range(total_ops):
        m.addConstr(g1[j] + g2[j] == 1)
        m.addConstr(gp.quicksum(x[j, i] for i in range(n)) == 1)
        m.addConstr(gp.quicksum(z[j, i] for i in range(n)) <= 2)
        for i in range(n):
            m.addConstr(x[j, i] <= z[j, i])
            m.addConstr(x[j, i] >= z[j, i] * epsilon)
            if i < 10:
                m.addConstr(z[j, i] <= g1[j])
            else:
                m.addConstr(z[j, i] <= g2[j])

    for i in range(n):
        m.addConstr(gp.quicksum(z[j, i] for j in range(total_ops)) <= 2)
        m.addConstr(throughput[i] == gp.quicksum(60 * x[j, i] / cycle_times[i] for j in range(total_ops)))

    avg_throughput = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="avg_throughput")
    m.addConstr(avg_throughput == gp.quicksum(throughput[i] for i in range(n)) / n)
    m.addConstr(avg_throughput >= mu_star - m.addVar(lb=0, name="throughput_dev"))

    for i in range(n):
        m.addConstr(deviation[i] == throughput[i] - avg_throughput)

    m.setObjective(gp.quicksum(deviation[i] * deviation[i] for i in range(n)), GRB.MINIMIZE)

    # Apply warm-start
    for j, i, xval, zval in warm_data:
        x[j, i].Start = xval
        z[j, i].Start = zval
        # Infer g1/g2 for warm-start
        if i < 10:
            g1[j].Start = 1
            g2[j].Start = 0
        else:
            g1[j].Start = 0
            g2[j].Start = 1

    return m, x, throughput, avg_throughput

# === SCHEDULE AND REPORT ===
def generate_schedule(x, throughput, avg_throughput):
    fractional_assignment = {
        j: {i: x[j, i].X for i in range(n) if x[j, i].X > epsilon}
        for j in range(total_ops)
    }

    total_units_per_hour = [0.0] * n
    operator_lines = []

    for j in range(total_ops):
        stations = sorted(fractional_assignment[j].items(), key=lambda x: -x[1])
        lines = [f"Operator {j+1}:"]
        lines.append("Hour | Primary Station | Units | Time (min) || Secondary Station | Units | Time (min)")

        if not stations:  # Handle case where no stations are assigned
            for h in range(1, shift_hours + 1):
                line = (
                    f" {h:<4}| "
                    f"{'-':^15} |"
                    f"{'-':>6} |"
                    f"{'-':>11} ||"
                    f"{'-':>16} |"
                    f"{'-':>6} |"
                    f"{'-':>9}"
                )
                lines.append(line)
            operator_lines.append("\n".join(lines))
            continue

        s1, f1 = stations[0]
        ct1 = cycle_times[s1]
        time_1 = 60 * f1
        units_1 = time_1 / ct1
        total_units_per_hour[s1] += units_1

        if len(stations) > 1:
            s2, f2 = stations[1]
            ct2 = cycle_times[s2]
            time_2 = 60 * f2
            units_2 = time_2 / ct2
            total_units_per_hour[s2] += units_2
        else:
            s2, time_2, units_2 = "-", "-", "-"

        for h in range(1, shift_hours + 1):
            line = (
                f" {h:<4}| "
                f"{s1+1:^15} |"
                f"{units_1:>6.2f} |"
                f"{time_1:>11.2f} ||"
                f"{('        -        ' if s2 == '-' else f' {s2+1:^16}'):>16}  |"
                f"{('   -  ' if units_2 == '-' else f'{units_2:>6.2f}'):>6} |"
                f"{(' -     ' if time_2 == '-' else f'{time_2:>9.2f}'):>9}"
            )
            lines.append(line)

        operator_lines.append("\n".join(lines))
        
    print("\n=== Station Summary ===")
    print(f"{'Station':<10} {'Units/Hr':>10}  {'Daily Units':>12}")
    print("-" * 35)
    for i in range(n):
        units_per_hour = total_units_per_hour[i]
        daily_units = throughput[i].X * shift_hours
        print(f"{i+1:<10} {units_per_hour:>10.2f} {daily_units:>12.2f}")

    print("\n=== Throughput Deviation Report ===")
    avg = avg_throughput.X
    for i in range(n):
        actual = throughput[i].X
        dev = actual - avg
        pct_dev = 100 * abs(dev) / avg if avg else 0
        flag = " <-- OUTLIER" if pct_dev > 10 else ""
        print(f"Station {i+1:2}: {actual:6.2f} units/hr (Δ {dev:+.2f}, {pct_dev:.1f}%) {flag}")
    print("\n=== Final Hourly Schedule ===")
    for block in operator_lines:
        print(block)
        print()

# === RUN MODEL ===
if __name__ == "__main__":
    mu_star, m1, x1, z1, g1, g2 = run_phase1()
    warm_data = extract_warm_start(m1, x1, z1, g1, g2)
    m2, x2, throughput2, avg_throughput2 = build_phase2_model(mu_star, warm_data)
    try:
        m2.optimize()
    except gp.GurobiError as e:
        print(f"Gurobi error in Phase 2: {e}")
        exit(1)

    if m2.status not in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        print(f"Phase 2 failed with status {m2.status} ({status_names.get(m2.status, 'UNKNOWN')})")
        exit(1)

    generate_schedule(x2, throughput2, avg_throughput2)

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 2703 rows, 1794 columns and 8619 nonzeros
Model fingerprint: 0xda372628
Variable types: 876 continuous, 918 integer (918 binary)
Coefficient statistics:
  Matrix range     [4e-02, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 60 rows and 60 columns
Presolve time: 0.03s
Presolved: 2643 rows, 1734 columns, 7650 nonzeros
Variable types: 850 continuous, 884 integer (884 binary)
Found heuristic solution: objective 17.7190713

Root relaxation: objective 1.882275e+01, 1170 iterations, 0.05 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 